In [ ]:
import json
import os
import pickle

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

In [2]:
with open("/openai/api_key.json") as f:
    config = json.load(f)


os.environ["OPENAI_API_KEY"] = config["key"]

In [3]:
SYSTEM_MESSAGE = SystemMessage("""
You are an expert in language and knowledgeable on how words are used.

Your task is to generate as many diverse definitions as possible for the a given word.  Follow these steps:

1. come up with a list of all possible parts of speech that the given word can be,e.g., noun, verb, adjective, etc.
2. for each part of speech, generate one or more examples of the given word for that parts of speech.  preappend the part of speech to the examples, e.g., "noun: example1", "verb: example2", etc.
3. combine all examples into a single list.

Return your response as a JSON object with the word as the key and the connotations as a list of strings.

example:

{
  "word": [
    "noun: example1", 
    "noun: example2", 
    "adjective: example3",]
}

""")

In [7]:
def ask_llm_for_solution(prompt, model="gpt-4o", temperature=0.7, max_tokens=4096):
   
    # Initialize the OpenAI LLM with your API key and specify the GPT-4o model
    llm = ChatOpenAI(
        model=model,
        temperature=temperature,
        max_tokens=max_tokens,
        model_kwargs={"response_format": {"type": "json_object"}},
    )


    conversation = [SYSTEM_MESSAGE, prompt]

    # Invoke the LLM
    response = llm.invoke(conversation)

    return response


In [ ]:
words = ["uphold", "discard", "honor", "energy", "state", "play", "justice", "labor", "pass", "fulfill", "draw", "keep", "blanket", "sham", "sheet", "throw"]

vocabulary = {}

for the_word in words:
    prompt = f"\n\ngiven word: {the_word}"
    prompt = HumanMessage(prompt)
    result = ask_llm_for_solution(prompt, temperature=1.0, max_tokens=4096)
    vocabulary[the_word] = json.loads(result.content)[the_word]

for word, definitions in vocabulary.items():
    print(f"\n\n{word}:")
    for definition in definitions:
        print(definition)




sham:
noun: A thing that is not what it is purported to be, for example, a lie, fraud, or hoax.
noun: A decorative pillow covering that is often used for ornamental purposes, such as a pillow sham.
verb: To falsely present something as the truth or significance, as in, 'to sham an illness.'
adjective: Describing something as fake or counterfeit, for example, 'a sham diamond ring.'


blanket:
noun: A large piece of fabric used for warmth, usually on a bed; e.g., a woolen blanket.
noun: A covering layer; e.g., a blanket of snow covered the town.
verb: To cover something completely; e.g., The city was blanketed in fog.
adjective: Applying to all instances or situations; e.g., a blanket ban on smoking in public places.


throw:
verb: to propel something with force through the air by a movement of the arm or hand
verb: to discard or get rid of something
verb: to cause someone to be in a particular state or condition
verb: to confuse or perplex someone
verb: to host or organize an event or

In [ ]:
# Save the vocabulary to a file
with open("vocabulary.pkl", "wb") as f:
    pickle.dump(vocabulary, f)